# 5. Live Camera Detection (ONNX) - Glute Bridge

**Objective:**
1.  Load the self-contained ONNX model.
2.  Initialize MediaPipe Pose for landmark detection.
3.  Capture video from a webcam.
4.  In a real-time loop, for each frame:
    a.  Extract pose landmarks.
    b.  Calculate geometric features (angles, distances).
    c.  Feed the features directly to the ONNX model for inference (scaling is handled internally).
    d.  Implement logic for state tracking, rep counting, and user feedback.
    e.  Display all information on the video frame.

**Note:** This notebook runs an OpenCV window. To stop the camera feed, press **'q'** while the window is active.

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import onnxruntime as ort
from collections import deque
import time
import os
import sys

# Add root project directory to sys.path
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.geometry_utils import GeometryUtils

## 5.1 Configuration and Constants

In [8]:
# --- Model and Paths ---
MODEL_ONNX_DIR = "../models/onnx/"
# >>>>> YOU CAN CHANGE THIS VALUE TO 'LR', 'KNN', 'DT', 'RF', or 'XGB' if you saved it earlier <<<<<
MODEL_TYPE_TO_LOAD = "RF"
ONNX_MODEL_PATH = os.path.join(MODEL_ONNX_DIR, f"{MODEL_TYPE_TO_LOAD}_model.onnx")

# --- Landmark & Feature Definitions ---
LANDMARK_NAMES = [
    'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow',
    'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index',
    'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle',
    'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'
]

FEATURE_COLUMN_NAMES = [
    'left_elbow_angle', 'right_elbow_angle', 'left_shoulder_angle', 'right_shoulder_angle', 
    'left_hip_angle', 'right_hip_angle', 'left_knee_angle', 'right_knee_angle', 
    'left_body_align_angle', 'right_body_align_angle', 'left_hip_deviation', 'right_hip_deviation', 
    'shoulder_hip_y_diff_left', 'shoulder_hip_y_diff_right', 'hip_ankle_y_diff_left', 'hip_ankle_y_diff_right',
    'torso_length_left', 'torso_length_right', 'leg_length_left', 'leg_length_right'
]

# --- Logic & Thresholds ---
VISIBILITY_THRESHOLD = 0.6
PREDICTION_SMOOTHING_WINDOW = 5
CONFIDENCE_THRESHOLD_UP = 0.70
CONFIDENCE_THRESHOLD_DOWN = 0.70
HOLD_TIME_GOAL = 1.5  # seconds to hold the 'up' pose
MIN_HOLD_FOR_REP = 0.5 # minimum time in 'up' pose to count as a rep

# --- Display Configuration ---
FONT = cv2.FONT_HERSHEY_SIMPLEX
COLOR_CORRECT = (0, 255, 0)
COLOR_INCORRECT = (0, 0, 255)
COLOR_WARNING = (0, 165, 255)
COLOR_NEUTRAL = (255, 255, 255)
COLOR_PROGRESS_BAR = (0, 255, 0)
COLOR_PROGRESS_BG = (100, 100, 100)

## 5.2 Load ONNX Model and Initialize Utilities

In [9]:
onnx_session = None
input_name = None
output_names = None

if not os.path.exists(ONNX_MODEL_PATH):
    print(f"FATAL: ONNX model not found at {ONNX_MODEL_PATH}")
    print("Please run Notebook 4 to convert the PKL model to ONNX.")
else:
    try:
        onnx_session = ort.InferenceSession(ONNX_MODEL_PATH)
        input_name = onnx_session.get_inputs()[0].name
        output_names = [output.name for output in onnx_session.get_outputs()]
        print("ONNX Model loaded successfully.")
        print(f"  Input Name: {input_name}")
        print(f"  Output Names: {output_names}")
    except Exception as e:
        print(f"Error loading ONNX model: {e}")
        onnx_session = None

# Initialize MediaPipe and Geometry Utilities
mp_pose = mp.solutions.pose
pose_detector = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
geo_utils = GeometryUtils()

ONNX Model loaded successfully.
  Input Name: float_input
  Output Names: ['output_label', 'output_probability']


## 5.3 Helper Functions for Live Detection

In [10]:
def extract_features_from_live_landmarks(landmarks):
    """Extracts features from live MediaPipe landmarks."""
    lm_coords = {name: [np.nan, np.nan] for name in LANDMARK_NAMES}
    for i, lm in enumerate(landmarks.landmark):
        if i < len(LANDMARK_NAMES) and lm.visibility > VISIBILITY_THRESHOLD:
            lm_coords[LANDMARK_NAMES[i]] = [lm.x, lm.y]

    # This inner function is just for cleaner code below
    def get_coord(name): return lm_coords.get(name, [np.nan, np.nan])
    
    # Identical feature extraction logic as in training
    features = {}
    features['left_elbow_angle'] = geo_utils.calculate_angle(get_coord('left_shoulder'), get_coord('left_elbow'), get_coord('left_wrist'))
    features['right_elbow_angle'] = geo_utils.calculate_angle(get_coord('right_shoulder'), get_coord('right_elbow'), get_coord('right_wrist'))
    features['left_shoulder_angle'] = geo_utils.calculate_angle(get_coord('left_elbow'), get_coord('left_shoulder'), get_coord('left_hip'))
    features['right_shoulder_angle'] = geo_utils.calculate_angle(get_coord('right_elbow'), get_coord('right_shoulder'), get_coord('right_hip'))
    features['left_hip_angle'] = geo_utils.calculate_angle(get_coord('left_shoulder'), get_coord('left_hip'), get_coord('left_knee'))
    features['right_hip_angle'] = geo_utils.calculate_angle(get_coord('right_shoulder'), get_coord('right_hip'), get_coord('right_knee'))
    features['left_knee_angle'] = geo_utils.calculate_angle(get_coord('left_hip'), get_coord('left_knee'), get_coord('left_ankle'))
    features['right_knee_angle'] = geo_utils.calculate_angle(get_coord('right_hip'), get_coord('right_knee'), get_coord('right_ankle'))
    features['left_body_align_angle'] = geo_utils.calculate_angle(get_coord('left_shoulder'), get_coord('left_hip'), get_coord('left_ankle'))
    features['right_body_align_angle'] = geo_utils.calculate_angle(get_coord('right_shoulder'), get_coord('right_hip'), get_coord('right_ankle'))
    features['left_hip_deviation'] = geo_utils.distance_point_to_line(get_coord('left_hip'), get_coord('left_shoulder'), get_coord('left_knee'))
    features['right_hip_deviation'] = geo_utils.distance_point_to_line(get_coord('right_hip'), get_coord('right_shoulder'), get_coord('right_knee'))
    ls_y, lh_y = get_coord('left_shoulder')[1], get_coord('left_hip')[1]
    features['shoulder_hip_y_diff_left'] = abs(ls_y - lh_y) if not (np.isnan(ls_y) or np.isnan(lh_y)) else np.nan
    rs_y, rh_y = get_coord('right_shoulder')[1], get_coord('right_hip')[1]
    features['shoulder_hip_y_diff_right'] = abs(rs_y - rh_y) if not (np.isnan(rs_y) or np.isnan(rh_y)) else np.nan
    la_y = get_coord('left_ankle')[1]
    features['hip_ankle_y_diff_left'] = abs(lh_y - la_y) if not (np.isnan(lh_y) or np.isnan(la_y)) else np.nan
    ra_y = get_coord('right_ankle')[1]
    features['hip_ankle_y_diff_right'] = abs(rh_y - ra_y) if not (np.isnan(rh_y) or np.isnan(ra_y)) else np.nan
    features['torso_length_left'] = geo_utils.calculate_distance(get_coord('left_shoulder'), get_coord('left_hip'))
    features['torso_length_right'] = geo_utils.calculate_distance(get_coord('right_shoulder'), get_coord('right_hip'))
    features['leg_length_left'] = geo_utils.calculate_distance(get_coord('left_hip'), get_coord('left_ankle'))
    features['leg_length_right'] = geo_utils.calculate_distance(get_coord('right_hip'), get_coord('right_ankle'))
    
    # Return as a numpy array in the correct order
    return np.array([features[name] for name in FEATURE_COLUMN_NAMES], dtype=np.float32).reshape(1, -1)

## 5.4 Main Video Processing Loop

In [11]:
def run_glute_bridge_detector():
    if onnx_session is None:
        print("Cannot run detector because the ONNX model is not loaded.")
        return

    cap = cv2.VideoCapture(0) # Use 0 for webcam
    if not cap.isOpened():
        print("Error: Cannot open webcam.")
        return

    # State variables
    pose_state = 'down'
    rep_counter = 0
    feedback = "Start in the down position"
    hold_start_time = None
    hold_duration = 0.0

    # For smoothing predictions
    proba_history = deque(maxlen=PREDICTION_SMOOTHING_WINDOW)
    
    print("Starting Glute Bridge Detector... Press 'q' to quit.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1) # Flip for selfie view
        h, w, _ = frame.shape

        # Pose detection
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_detector.process(image_rgb)

        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Extract features
            live_features = extract_features_from_live_landmarks(results.pose_landmarks)
            
            # Check if features are valid before prediction
            if np.isnan(live_features).sum() > len(FEATURE_COLUMN_NAMES) * 0.5:
                feedback = "Body not fully visible"
                proba_history.clear()
            else:
                
                # Inference
                onnx_outputs = onnx_session.run(output_names, {input_name: live_features})
                # onnx_outputs[1] is the probability output.
                prob_output = onnx_outputs[1] 
                
                # The output can be a list of dicts [{0: prob, 1: prob}] or an array [[prob, prob]].
                # We need to parse it into a simple numpy array.
                if isinstance(prob_output, list) and len(prob_output) > 0 and isinstance(prob_output[0], dict):
                    prob_dict = prob_output[0]
                    # Create array ordered by class index (0, 1)
                    probabilities = np.array([prob_dict.get(0, 0.0), prob_dict.get(1, 0.0)], dtype=np.float32)
                elif isinstance(prob_output, np.ndarray):
                    # Handles the case where output is already like [[0.1, 0.9]]
                    probabilities = prob_output[0]
                else:
                    # Fallback for unexpected formats
                    print(f"Warning: Unexpected ONNX probability format: {type(prob_output)}")
                    probabilities = np.array([0.5, 0.5]) # Default to uncertainty

                # Now, 'probabilities' is guaranteed to be a numpy array
                proba_history.append(probabilities)

                if len(proba_history) == PREDICTION_SMOOTHING_WINDOW:
                    smoothed_proba = np.mean(proba_history, axis=0) # This will now work
                    pred_label = np.argmax(smoothed_proba)
                    confidence = smoothed_proba[pred_label]

                    # State machine for rep counting
                    if pose_state == 'down':
                        if pred_label == 1 and confidence > CONFIDENCE_THRESHOLD_UP: # Transition to UP
                            pose_state = 'up'
                            hold_start_time = time.time()
                            feedback = "Hold it..."
                        else:
                            feedback = "Lift your hips"
                            hold_duration = 0.0
                            hold_start_time = None
                            
                    elif pose_state == 'up':
                        if pred_label == 0 and confidence > CONFIDENCE_THRESHOLD_DOWN: # Transition to DOWN
                            if hold_duration >= MIN_HOLD_FOR_REP:
                                rep_counter += 1
                                feedback = f"Rep {rep_counter} counted!"
                            else:
                                feedback = "Hold longer next time!"
                            pose_state = 'down'
                            hold_duration = 0.0
                            hold_start_time = None
                        else: # Still in UP state
                            hold_duration = time.time() - hold_start_time
                            if hold_duration >= HOLD_TIME_GOAL:
                                feedback = "Great hold! Lower slowly."
                            else:
                                feedback = f"Holding... {hold_duration:.1f}s"
        else:
            feedback = "No person detected"
            proba_history.clear()

        # --- Display UI --- 
        # Rep Counter Box
        cv2.rectangle(frame, (0, 0), (250, 80), (20, 20, 20), -1)
        cv2.putText(frame, 'REPS', (15, 30), FONT, 0.8, COLOR_NEUTRAL, 2, cv2.LINE_AA)
        cv2.putText(frame, str(rep_counter), (100, 70), FONT, 2, COLOR_NEUTRAL, 3, cv2.LINE_AA)

        # Feedback Box
        cv2.rectangle(frame, (250, 0), (w, 80), (20, 20, 20), -1)
        cv2.putText(frame, 'FEEDBACK', (265, 30), FONT, 0.8, COLOR_NEUTRAL, 2, cv2.LINE_AA)
        feedback_color = COLOR_CORRECT if 'Great' in feedback or 'counted' in feedback else COLOR_WARNING
        cv2.putText(frame, feedback, (265, 65), FONT, 1, feedback_color, 2, cv2.LINE_AA)

        # Hold Progress Bar
        if pose_state == 'up':
            progress = min(1.0, hold_duration / HOLD_TIME_GOAL)
            bar_width = int(progress * (w - 20))
            cv2.rectangle(frame, (10, h - 30), (w - 10, h - 10), COLOR_PROGRESS_BG, -1)
            cv2.rectangle(frame, (10, h - 30), (10 + bar_width, h - 10), COLOR_PROGRESS_BAR, -1)

        # Display Frame
        cv2.imshow('Glute Bridge Detector', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    # This is important for Jupyter to properly release the window
    for i in range(5):
        cv2.waitKey(1)
    print("Detector stopped.")

In [12]:
# Run the main loop
run_glute_bridge_detector()

Starting Glute Bridge Detector... Press 'q' to quit.
Detector stopped.
